In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

In [ ]:
if not os.path.exists("details.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/13_CCL/details.png

In [ ]:
srubki = cv2.imread('details.png', cv2.IMREAD_GRAYSCALE)
plt.axis('off')
plt.imshow(srubki,'gray')
plt.show()

In [ ]:
#Element strukturalny
struktura_1 = cv2.getStructuringElement(cv2.MORPH_RECT,(15,15))

#Operacje top-hat i bottom-hat
TOP_HAT=cv2.morphologyEx(srubki, cv2.MORPH_TOPHAT, struktura_1)
BOTTOM_HAT=cv2.morphologyEx(srubki, cv2.MORPH_BLACKHAT, struktura_1)

fig, axs = plt.subplots(1, 2)
fig.set_size_inches(20, 10)
plt.subplot(121),plt.imshow(TOP_HAT, 'gray', vmin=0, vmax=256),plt.title('OPERACJA TOP-HAT')
plt.axis('off')
plt.subplot(122),plt.imshow(BOTTOM_HAT, 'gray', vmin=0, vmax=256),plt.title('OPERACJA BOTTOM-HAT')
plt.axis('off')

plt.show()


In [ ]:
#Doprowadzenie do postaci binarnej
obraz_binarny=np.where(BOTTOM_HAT <60, 0, 1)
obraz_binarny=obraz_binarny.astype('uint8')
plt.imshow(obraz_binarny,'gray')
plt.axis('off')
plt.show()


In [ ]:
obraz_binarny_1=np.where(srubki < 220, 0, 1)
obraz_binarny_1=obraz_binarny_1.astype('uint8')
plt.imshow(obraz_binarny_1,'gray')
plt.axis('off')
plt.show()

In [ ]:
suma=obraz_binarny+obraz_binarny_1
suma=suma.astype('uint8')
plt.imshow(suma,'gray')
plt.axis('off')
plt.show()

suma_1=np.where(suma > 0, 1, 0)
suma_1=suma_1.astype('uint8')
suma_1=cv2.medianBlur(suma_1, 3)   

#dillation
kernel = np.ones((3,3))
dilate_1=cv2.dilate(suma_1,kernel,iterations=1)
plt.imshow(dilate_1,'gray')
plt.axis('off')
plt.show()

In [ ]:
#Wyliczenie statystyki
labels,tresh,stats,centroids = cv2.connectedComponentsWithStats(dilate_1)
plt.imshow(tresh,'gray')
plt.axis('off')
plt.show()

print("współrzędne każdego połączonego komponentu")
print(int(centroids[0,0]))

In [ ]:
#kopia iobrazu indeksowanego
ccl_vis = tresh.copy()
s=0
for label_num in range(2,labels+1):
    cv2.putText(ccl_vis,str(label_num-1),(int(centroids[label_num-1,0]),int(centroids[label_num-1,1])),cv2.FONT_HERSHEY_SIMPLEX,1,(20,0,0),2)
plt.imshow(ccl_vis,'gray')
plt.axis('off')
plt.show()

In [ ]:
# momens_real = []
# hu_momens_real = []
# images_real = []
# for i in range(1,labels):
#     print("Numer obiektu to: {}".format(i))
#     I = (I_indeks == i)*1
#     I = I.astype('uint8')
#     images_real.append(I)
#     plt.imshow(I, 'gray')
#     plt.show()
#     momens_real.append(cv2.moments(I))
#     hu_momens_real.append(cv2.HuMoments(momens_real[i-1]))
#     print(momens_real[i-1]['m00'])
#     print(hu_momens_real[i-1][0][0])
#     print(hu_momens_real[i-1][1][0])
#     print(hu_momens_real[i-1][2][0])

moment_geometryczny_1 = []
momentHU_1 = []
obraz_1 = []

for i in range(1,labels):
    print("Numer obiektu to: {}".format(i))
    #wybranie obiektu
    I = (tresh == i)*1
    I = I.astype('uint8')
    #wyświetlenie obiektu
    obraz_1.append(I)
    plt.imshow(I, 'gray')
    plt.show()
    #obliczenie momentu geometrycznego
    moment_geometryczny_1.append(cv2.moments(I))
    #obliczenie momentu HU
    momentHU_1.append(cv2.HuMoments(moment_geometryczny_1[i-1]))
    
    print("Moment geometryczny dla {}".format(i))
    print(moment_geometryczny_1[i-1]['m00'])
    print()
    print("Moment Hu dla {}".format(i))
    print(momentHU_1[i-1][0][0])

In [ ]:
A , B = tresh.shape
new_1 = np.zeros((A,B))
for i in range(len(obraz_1)):
    if(moment_geometryczny_1[i]['m00']>550 and moment_geometryczny_1[i]['m00']<980 and momentHU_1[i][0][0] > 0.2 and momentHU_1[i][1][0]<0.6):
        new_1 += obraz_1[i]
plt.imshow(new_1, 'gray')
plt.show()

In [ ]:
A , B = tresh.shape
new_2 = np.zeros((A,B))
for i in range(len(obraz_1)):
    if(moment_geometryczny_1[i]['m00']>320 and moment_geometryczny_1[i]['m00']<920 and momentHU_1[i][0][0] > 0.17 and momentHU_1[i][0][0] < 0.19): 
        new_2 += obraz_1[i]
plt.imshow(new_2, 'gray')
plt.show()

In [ ]:
A , B = tresh.shape
new_3 = np.zeros((A,B))
for i in range(len(obraz_1)):
    if(moment_geometryczny_1[i]['m00']>260 and moment_geometryczny_1[i]['m00']<970 and momentHU_1[i][0][0] > 0.19 and momentHU_1[i][0][0] < 0.25):
        new_3 += obraz_1[i]
plt.imshow(new_3, 'gray')
plt.show()